In [28]:
import pandas as pd

def calculate_percentage(fee, SalePrice, stock):
    if SalePrice == 0:
        return 0
    percentage = (fee / SalePrice) * 100
    portfolio = pd.read_csv('portfolio.csv')
    
    # Update the SalePrice for the given stock on the most recent day
    portfolio.loc[(portfolio['Stock Name'] == stock) & (portfolio['Day'] == portfolio['Day'].max()), 'SalePrice'] = SalePrice - fee
    
    # Write to the log file in append mode
    with open('Log.txt', 'a') as file:
        file.write(f"{stock}, sold at {SalePrice - fee}, fee: {fee}, percentage: {percentage}\n")
    
    # Print the log message
    print(f"{stock}, sold at {SalePrice - fee}, fee: {fee}, percentage: {percentage}")
    
    return portfolio

# Example usage
fee = 0.02
SalePrice = 124.13
stock = "AMZN"
Sale = calculate_percentage(fee, SalePrice, stock).fillna(0)

AMZN, sold at 124.11, fee: 0.02, percentage: 0.016112140497865142


In [32]:
portfolio = pd.read_csv('portfolio2.csv').sort_values(by='Day')
portfolio.to_csv('portfolio2.csv', index=False)

In [ ]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['Cash'] = portfolio[portfolio['Day'] == portfolio['Day'].max()]['SalePrice'].sum()
